In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://thumbor.forbes.com/thumbor/fit-in/900x510/https://www.forbes.com/advisor/wp-content/uploads/2020/06/pexels-pixabay-164571-scaled-e1602462314356.jpg)

The term credit card churning was used to refer to applying for the same type of credit card over and over again. This is done primarily to collect a large welcome bonus available on a card, then cancel the card once that bonus has been earned.

Credit card churners are some of the bank’s least profitable customers.

Credit score is likely to drop slightly with each card one applies for.

> *Note here the inherent tradeoff for customers while indulging in credit card churning.*

Banks try to prevent churning by mean of targeted offers. A targeted offer may come with a higher welcome bonus than the standard offer for a card or it may allow you to get a card that you otherwise wouldn’t be eligible for.

> *Note here that while predicting for churn, it is important to have both a good precision and recall score to maintain profitability.*


Source: https://www.forbes.com/advisor/credit-cards/what-is-credit-card-churning/

Importing Data

In [ ]:
data = pd.read_csv('/kaggle/input/credit-card-customers/BankChurners.csv')

Encoding Categorical Data

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
X = pd.DataFrame(np.vstack([data['Customer_Age'], 
                            encoder.fit_transform(data['Gender']), 
                            data['Dependent_count'],#This item is quantitative, hence no need for encoding
                            encoder.fit_transform(data['Education_Level']),
                            encoder.fit_transform(data['Marital_Status']),
                            encoder.fit_transform(data['Income_Category']),
                            encoder.fit_transform(data['Card_Category'])]).T, columns=['Customer_Age', 'Gender', 'Dependent_count','Education_Level','Marital_Status','Income_Category','Card_Category'])

y = pd.DataFrame(np.vstack([encoder.fit_transform(data['Attrition_Flag'])]).T, columns=['Attrition_Flag'])

Appending Quantitative Data to Encoded Data

In [ ]:
X = pd.DataFrame(np.hstack([X,data.iloc[:,9:-2]]), columns=['Customer_Age', 'Gender',
'Dependent_count', 'Education_Level', 'Marital_Status',
'Income_Category', 'Card_Category', 'Months_on_book',
'Total_Relationship_Count', 'Months_Inactive_12_mon',
'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio'])

Scaling All Features

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

Preparing Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100)

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.4)

Optimizing Model Parameters

In [ ]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(model,
                 {'max_features': np.arange(.05, 1, .05),
                  'min_samples_leaf': np.arange(1, 50, 5)},
                  cv=5,
                  n_jobs=4,
                  scoring='neg_mean_squared_error',
                  verbose=True)

gs.fit(Xtrain, ytrain)

Accuracy

In [ ]:
model = gs.best_estimator_

model.score(Xtest, ytest)

Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

cm = confusion_matrix(ytest, model.predict(Xtest))

disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=['Attrited','Existing'])

disp = disp.plot()

plt.show()

Classification Report

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(ytest, model.predict(Xtest), target_names=['Attrited', 'Existing']))

Feature Importance

In [ ]:
coeff_df=pd.DataFrame(model.feature_importances_.T, X.columns, columns=['Importance']).sort_values(by=['Importance'],ascending=False)
coeff_df.plot.bar()